# MNIST with CNN using Keras

## Setup

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

import numpy as np
import boto3
import time

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./.data/', one_hot=True)

Using TensorFlow backend.


Extracting ./.data/train-images-idx3-ubyte.gz
Extracting ./.data/train-labels-idx1-ubyte.gz
Extracting ./.data/t10k-images-idx3-ubyte.gz
Extracting ./.data/t10k-labels-idx1-ubyte.gz


In [2]:
X_train = mnist.train.images
X_test = mnist.test.images
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')/255
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')/255

y_train = mnist.train.labels
y_test = mnist.test.labels

## Define network

In [3]:
model = Sequential()
model.add(Convolution2D(32, 5, 5, activation='relu', input_shape=(1,28,28), dim_ordering='th'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, 5, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

## Train

In [4]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [5]:
t0 = time.time()
model.fit(X_train, y_train,
          batch_size=50,
          nb_epoch=10
         )
print "training time = {}".format(time.time() - t0)

Epoch 1/10
55000/55000 [==============================] - 17s - loss: 1.1609 - acc: 0.5838    
Epoch 2/10
55000/55000 [==============================] - 16s - loss: 0.1894 - acc: 0.9422    
Epoch 3/10
55000/55000 [==============================] - 15s - loss: 0.1222 - acc: 0.9619    
Epoch 4/10
55000/55000 [==============================] - 15s - loss: 0.0939 - acc: 0.9703    
Epoch 5/10
55000/55000 [==============================] - 15s - loss: 0.0781 - acc: 0.9750    
Epoch 6/10
55000/55000 [==============================] - 15s - loss: 0.0680 - acc: 0.9779    
Epoch 7/10
55000/55000 [==============================] - 16s - loss: 0.0591 - acc: 0.9811    
Epoch 8/10
55000/55000 [==============================] - 16s - loss: 0.0530 - acc: 0.9835    
Epoch 9/10
55000/55000 [==============================] - 15s - loss: 0.0476 - acc: 0.9847    
Epoch 10/10
55000/55000 [==============================] - 15s - loss: 0.0438 - acc: 0.9857    
training time = 160.831033945


## Evaluate

In [6]:
score = model.evaluate(X_test, y_test, verbose=1)
print score

 9696/10000 [============================>.] - ETA: 0s[0.044591301128594205, 0.98470000000000002]


## Save Model

In [7]:
model.save('./keras.hdf5')

s3 = boto3.client('s3')
s3.upload_file(
    './keras.hdf5',
    'jakechenawstemp',
    'keras.hdf5'
)